# Search Application

In [1]:
# importing required libraries
import psycopg2
from pymongo import MongoClient
from fastapi import HTTPException
from exceptions import *
import requests

## Connecting to the Databases Storing the Information

In [3]:
# connecting to the PostgreSQL database
try:
    p_conn = psycopg2.connect(
        dbname = "twitter",
        user = "varshiniyanamandra",
        password = "",
        host = "localhost",
        port = "5432"
    )
except psycopg2.OperationalError as e:
    # raise an error if the connection is unsuccessful
    print(f"Unable to connect to PostgreSQL: {e}")

# opening a cursor to perform database operations
p_cur = p_conn.cursor()

# print the PostgreSQL server information
print(p_conn.get_dsn_parameters(), "\n")

{'user': 'varshiniyanamandra', 'passfile': '/Users/varshiniyanamandra/.pgpass', 'channel_binding': 'prefer', 'dbname': 'twitter', 'host': 'localhost', 'port': '5432', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'sslsni': '1', 'ssl_min_protocol_version': 'TLSv1.2', 'gssencmode': 'prefer', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'} 



In [ ]:
# connect to the MongoDB database
mongo_conn = MongoClient('mongodb://localhost:27017/')
mongo_db = mongo_conn['twitter_data']
tweets_collection = mongo_db['tweets']

## Building the Search Application

### Defining Utility Functions

In [ ]:
# function to get user information
def get_user_info(user_id: str):
    """
        This function returns the user information as a JSON object.
        Input:
            user_id (str): Twitter user ID which we want to look up
        Output:
            user_out (JSON object): user information corresponding to user_id
    """
    p_cur.execute("SELECT * FROM TwitterUser WHERE id = {0};".format(user_id))
    user_info = p_cur.fetchone()
    if user_info is None:
        # raise an exception if the user doesn't exist in the database
        raise HTTPException(status_code = UserNotFoundError.code, detail = UserNotFoundError.description)
    user_out = {
        'id': user_info[0],
        'name': user_info[1],
        'screen_name': user_info[2],
        'location': user_info[3],
        'created_at': user_info[4],
        'followers_count': user_info[5],
        'friends_count': user_info[6],
        'statuses_count': user_info[7],
        'favorites_count': user_info[8]
    }

    return user_out

In [ ]:
# function to retreive tweets containing a specified keyword
def retrieve_tweets_keyword(keyword: str, sort_criterion = None):
    """
        Function to get the information of tweets based on a user-specified keyword.
        Input:
            keyword (str): user-specified keyword
            sort_criterion (str): criteria for sorting the results
                default: decreasing order of popularity (favorite count)
                valid inputs:
                    'oldestToNewest', 'newestToOldest', 'popularity'
        Output:
            out (list): list of tweets containing the keyword
    """

    # check if sort_criterion is valid, if specified:
    if sort_criterion is not None:
        if sort_criterion not in ['oldestToNewest', 'newestToOldest', 'popularity']:
            raise HTTPException(status_code = InvalidSortCriterionError.code, detail = InvalidSortCriterionError.description)

    out = []
    query = {'$text': {'$search': keyword}}
    tweets_match = tweets_collection.find(query) # we can add .limit(PAGE_LIMIT) here, if needed
    for result in tweets_match:
        tweet = {
            'id': result['_id'],
            'text': result['text'],
            'user_id': result['user_id'],
            'quote_count': result['quote_count'],
            'reply_count': result['reply_count'],
            'retweet_count': result['retweet_count'],
            'favorite_count': result['favorite_count'],
            'created_at': result['timestamp'],
            'coordinates': result['coordinates']
        }
        # add information on whether the tweet is a retweet
        if 'retweet' in result:
            tweet['retweet'] = "Yes"
        else:
            tweet['retweet'] = "No"

        
        out.append(tweet)

    # sort the results from oldest to newest before returning, if specified 'oldestToNewest'
    if sort_criterion == "oldestToNewest":
        return out.sort(key = lambda x: int(x['timestamp_ms']), reverse = False)
    elif sort_criterion == "newestToOldest":
        # otherwise sort the results from newest to oldest before returning if specified 'newestToOldest'
        return out.sort(key = lambda x: int(x['timestamp_ms']), reverse = True)
    else:
        # sort the output in the decreasing order of favorites (popularity), by default or if specified 'popularity'
        return out.sort(key = lambda x: int(x['favorite_count']), reverse = True)

In [ ]:
# function to search tweets based on tweet id
def retrieve_tweet(tweet_id: str):
    """
        Function to get the information of a tweet based on a user-specified tweet ID.
        Input:
            tweet_id (str): user-specified tweet ID
        Output:
            tweet (JSON object): tweet corresponding to tweet_id
    """
    query = {'id': tweet_id}
    result = tweets_collection.find_one(query)
    if result.count() == 0:
        # raise an exception if the tweet doesn't exist in the database
        raise HTTPException(status_code = TweetNotFoundError.code, detail = TweetNotFoundError.description)
    tweet = {
        'id': result['_id'],
        'text': result['text'],
        'user_id': result['user_id'],
        'quote_count': result['quote_count'],
        'reply_count': result['reply_count'],
        'retweet_count': result['retweet_count'],
        'favorite_count': result['favorite_count'],
        'created_at': result['timestamp'],
        'coordinates': result['coordinates']
    }
    # add information on whether the tweet is a retweet
    if 'retweet' in result:
        tweet['retweet'] = "Yes"
    else:
        tweet['retweet'] = "No"

    return tweet

In [ ]:
# function to retrieve all tweets by a user
def retrieve_tweets_user(user_id: str, sort_criterion = 'popularity'):
    """
        Function to retrieve all tweets by a specific user (user-specified user ID)
        Input:
            user_id (str): user-specified user ID
            sort_criterion (str): criteria for sorting the results
                default: decreasing order of popularity (favorite count)
                valid inputs:
                    'oldestToNewest', 'newestToOldest', 'popularity'
        Output:
            tweets_list (list): list of tweets made by a user
    """
    # check if the user id is valid
    p_cur.execute("SELECT id FROM TwitterUser WHERE id = {0};".format(user_id))
    user_id_db = p_cur.fetchone()
    if user_id_db is None:
        # raise an exception if the user doesn't exist in the database
        raise HTTPException(status_code = UserNotFoundError.code, detail = UserNotFoundError.description)

    # if the user exists, proceed to search MongoDB
    query = {'user_id': user_id}
    tweets_match = tweets_collection.find(query)
    
    if tweets_match.count() == 0:
        return "This user has not tweeted anything yet."

    tweets_list = []
    for result in tweets_match:
        tweet = {
            'id': result['_id'],
            'text': result['text'],
            'user_id': result['user_id'],
            'quote_count': result['quote_count'],
            'reply_count': result['reply_count'],
            'retweet_count': result['retweet_count'],
            'favorite_count': result['favorite_count'],
            'created_at': result['timestamp'],
            'coordinates': result['coordinates']
        }
        # add information on whether the tweet is a retweet
        if 'retweet' in result:
            tweet['retweet'] = "Yes"
        else:
            tweet['retweet'] = "No"

        tweets_list.append(tweet)

    # sort the results from oldest to newest before returning, if specified 'oldestToNewest'
    if sort_criterion == "oldestToNewest":
        return tweets_list.sort(key = lambda x: int(x['timestamp_ms']), reverse = False)
    elif sort_criterion == "newestToOldest":
        # otherwise sort the results from newest to oldest before returning if specified 'newestToOldest'
        return tweets_list.sort(key = lambda x: int(x['timestamp_ms']), reverse = True)
    else:
        # sort the output in the decreasing order of favorites (popularity), by default or if specified 'popularity'
        return tweets_list.sort(key = lambda x: int(x['favorite_count']), reverse = True)

In [ ]:
# function to retrieve tweets based on location
def retrieve_tweets_location(location: str, distance = 100000, sort_criterion = 'popularity'):
    """
        Function to retrieve tweets near a specified location.
        Input:
            location (str): user-specified location
            distance (int): radius of the search (100 kilometers, by default)
            sort_criterion (str): criteria for sorting the results
                default: decreasing order of popularity (favorite count)
                valid inputs:
                    'oldestToNewest', 'newestToOldest', 'popularity'
        Output:
            tweets_list (list): list of tweets made from within the radius of the specified location
    """
    # getting the latitude and longitude of the location specified
    endpoint = "https://nominatim.openstreetmap.org/search"
    params = {"q": location, "format": "json", "limit": 1}
    # sending a request of the Nominatim API
    response = requests.get(endpoint, params=params)
    result = response.json()[0]
    # getting the latitude and longitude from the response of the API
    latitude = result["lat"]
    longitude = result["lon"]

    # creating a geospatial index on the coordinates field
    tweets_collection.create_index([("coordinates", "2dsphere")])
    
    query = {"coordinates": {"$near": {"$geometry": {"type": "Point", "coordinates": [longitude, latitude]}, "$maxDistance": distance}}}
    tweets_match = tweets_collection.find(query)
    
    if tweets_match.count() == 0:
        return "There are no tweets near this location yet."

    tweets_list = []
    for result in tweets_match:
        tweet = {
            'id': result['_id'],
            'text': result['text'],
            'user_id': result['user_id'],
            'quote_count': result['quote_count'],
            'reply_count': result['reply_count'],
            'retweet_count': result['retweet_count'],
            'favorite_count': result['favorite_count'],
            'created_at': result['timestamp'],
            'coordinates': result['coordinates']
        }
        # add information on whether the tweet is a retweet
        if 'retweet' in result:
            tweet['retweet'] = "Yes"
        else:
            tweet['retweet'] = "No"

        tweets_list.append(tweet)

    # sort the results from oldest to newest before returning, if specified 'oldestToNewest'
    if sort_criterion == "oldestToNewest":
        return tweets_list.sort(key = lambda x: int(x['timestamp_ms']), reverse = False)
    elif sort_criterion == "newestToOldest":
        # otherwise sort the results from newest to oldest before returning if specified 'newestToOldest'
        return tweets_list.sort(key = lambda x: int(x['timestamp_ms']), reverse = True)
    else:
        # sort the output in the decreasing order of favorites (popularity), by default or if specified 'popularity'
        return tweets_list.sort(key = lambda x: int(x['favorite_count']), reverse = True)